In [6]:
from keras.models import load_model
from keras.preprocessing.image import img_to_array
import numpy as np
import cv2


In [7]:
classifier = load_model('model_v6_23.hdf5')

In [8]:
classifier.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_54 (Conv2D)           (None, 46, 46, 32)        320       
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 44, 44, 64)        18496     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 8, 8, 128)         147584    
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 4, 4, 128)       

In [9]:
labels={0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
cap=cv2.VideoCapture(0)
font=cv2.FONT_HERSHEY_SIMPLEX
while True:
    _,img=cap.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #cv2.imshow("image",img)
    faces=face_cascade.detectMultiScale(img,1.1,5)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),3)
        if len(faces)==0:
            text="Sorry, No Face Detected"
            img=cv2.putText(img,text,(0,50),font,1,(0,0,255),1,cv2.LINE_AA)
        elif len(faces)>1:
            text="Multiple Faces Detected"
            img=cv2.putText(img,text,(0,50),font,1,(0,0,255),1,cv2.LINE_AA)
        else:
            roi=img[y:y+h,x:x+w]
            roi=cv2.resize(roi,(48,48))
            gray=cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
            roi = gray.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            preds = classifier.predict(roi)[0]
            label_img = labels[preds.argmax()] 
            text="Emotion:"+label_img
            #print(preds)
            img=cv2.putText(img,text,(x,y-10),font,1,(0,0,255),1,cv2.LINE_AA)
    cv2.imshow("image",img)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()        

cv2.destroyWindow("image")
